## Connect to ICW:

In [1]:
import cadspy

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

User number u243700
Enter Password:  ··············


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

Below are some packages to get you started. You don't have to use them but you may find them useful!

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<br>

### Data

#### S19 Lounge Eligibility data

In [5]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [6]:
# df_lounge_eligibility.head(2)

In [7]:
# df_lounge_eligibility.shape

#### S19 Flight info

In [8]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [9]:
# df_flight_info.head(2)

#### Station Code Decode

In [10]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [11]:
# df_country.head(2)

#### Additional Station Decodes from ICW reference table

There are lots of destinations which are in lounge_elig but not in df_country.
They are all new (since ~2019) routes at Heathrow.

We can access this information from an ICW table. We load that ICW reference table and filter for results not in our merged table, but that are in lounge_elig.

This should result in zero nan values arising from missing country information

In [12]:
query = """
SELECT STN_CD, COUNTRY_CD, COUNTRY_NM, CORP_GEOG_CTRY_GRP_NM, CORP_GEOG_CONTINENT_NM
FROM REF_GEOG_LOC_HIERARCHY
"""
df_additional_country_decodes=icw.queryToDataframe(query)

#### Aircraft Type

In [13]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [14]:
# df_acft_typ.head(2)

In [15]:
# df_acft_typ.shape

<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [16]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

In [17]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)
format_2 = headers_and_first_row(df_flight_info)

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ, df_additional_country_decodes).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [18]:
# Stripping strings.
# Define the function to strip strings from the tables.

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

In [19]:
tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_additional_country_decodes]
for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

Clean the flight info dataset.

In [20]:
# Removing duplucates of flight_info
# Sort the table
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated.
# Duplicated flights arise when e.g. flights return to LHR. We want to keep the flight as originally planned as this is represented in the lounge eligibility dataset.
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")

In [21]:
# Create separate columns for date and time within df_flight_info.

df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

Clean the country information dataset.

In [22]:
# Create separate columns for the departure and arrival airports. This is needed when joining to lounge eligibility.
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)

Clean the aircraft type dataset

In [23]:
# Change seat quantities to integers.
df_acft_typ['FIRST_SEATS_QTY'] = df_acft_typ['FIRST_SEATS_QTY'].astype('int')
df_acft_typ['FIRST_SEATS_QTY'] = df_acft_typ['CLUB_SEATS_QTY'].astype('int')
df_acft_typ['FIRST_SEATS_QTY'] = df_acft_typ['PREM_ECONOMY_SEATS_QTY'].astype('int')
df_acft_typ['FIRST_SEATS_QTY'] = df_acft_typ['ECONOMY_SEATS_QTY'].astype('int')



df_acft_typ.head(1)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,132,24,0,132


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

In [24]:
# We first merge flight information onto the lounge eligibility dataset.
# We merge on three common characteristics: Operating Airline Code, flight number and departure date.
# We use left merge: this retains all lounge_elig rows are retained, albeit with NaNs.

df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )
# display the merged table.
df_lounge_elig_flight_info.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00


Get rows with any null values

In [25]:
print(df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].shape)
print(len(df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)]['Skew_Id'].unique()))

# There are currently 1183 rows for which flight information is not available.
# This represents 357 missing flights.

(1183, 25)
357


In [26]:
# # Example of missing flight number.
# print(df_flight_info[df_flight_info['OPG_FLT_NO'] == 8642].head(5))
# print(df_lounge_elig_flight_info[df_lounge_elig_flight_info['OPERATING_FLT_NO']==8642]['Skew_Id'].unique()) # example of all days with missing flight info for given flight number.

### Merge `df_country` to table

Begin with cleaning the additional country decode information.

In [27]:
# For efficiency, we should merge df_country and df_additional_country_decodes first.
# Firstly, organise df_additional_country_decodes to have the same columns.
# if statement to avoid throwing errors if column renaming from cell below has already been done.

if 'STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'STN_CD'
elif 'ARR_STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'ARR_STN_CD'
df_additional_country_decodes = df_additional_country_decodes[~df_additional_country_decodes[station_code_col].isin(df_country['ARR_STN_CD'])]

# for more efficiency, we can remove all those rows which are not needed as there are no lounge elig rows with that destinations.

df_additional_country_decodes = df_additional_country_decodes[df_additional_country_decodes[station_code_col].isin(df_lounge_elig_flight_info['DISCHARGE_STN_CD'])]
print(df_additional_country_decodes.shape)
# This leaves 30 destinations.

(30, 5)


In [28]:
# We need df_additional_country_decodes to have the same column names and order as df_country, ready for pd.concat.
# By merging now, we eliminate the need to merge separately with df_lounge_elig.

df_additional_country_decodes['ROUTE'] = 'LHR' + df_additional_country_decodes['STN_CD']
df_additional_country_decodes['DEP_STN_CD'] = 'LHR'

# We must ensure column names and order of df_additional_country_decodes match df_country.

df_additional_country_decodes.rename(columns={'STN_CD':'ARR_STN_CD'},inplace=True)
df_additional_country_decodes = df_additional_country_decodes[df_country.columns]

In [29]:
# concat the two country tables together.
df_country = pd.concat([df_country,df_additional_country_decodes])

In [30]:
# merge country info to the combined dataset. 
# We merge on a column originally derived from lounge_elig. This ensures there are no missing values due to missing flight information.

df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['DISCHARGE_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['ARR_STN_CD'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

# df_lounge_elig_flight_info = df_lounge_elig_flight_info.drop_duplicates()

df_lounge_elig_flight_info_country.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI


In [31]:
df_lounge_elig_flight_info_country.shape

(1534750, 32)

Check for null values

In [32]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country['CORP_GEOG_CONTINENT_NM'].isna()].head(5)
# There are no null values arising from lack of country data.

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD


### `df_acft_typ`

In [33]:
# merge aircraft type into the combined country/flight info/lounge_elig dataset.
# We have to merge on a flight_info column. This means all rows with missing flight information will not get aircraft type information.

df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,113.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,113.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,150.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,119.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,212.0,45.0,0.0,212.0


In [34]:
df_lounge_elig_flight_info_country_acft_typ.shape

(1534750, 37)

Check for null

In [35]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
300,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,SAPP,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
466,2019-09-298598,BA,8598,2019-09-29,LHR,CFU,M,M,,,Not eligible,43,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRCFU,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,CFU,NaN,NaN,NaN,NaN,NaN
876,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
1452,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,,,Not eligible,144,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
11711,2019-06-168586,BA,8586,2019-06-16,LHR,KLX,M,M,Gold,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRKLX,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,KLX,NaN,NaN,NaN,NaN,NaN


In [36]:
df_lounge_elig_flight_info_country_acft_typ.loc[(df_lounge_elig_flight_info_country_acft_typ['IATA_AC_TYP_CD'].notna())&(df_lounge_elig_flight_info_country_acft_typ['WB_NB_CAT'].isna())]['ACT_AC_TYP_CD'].unique()

# We are missing two aircraft types: T7 and U7.

# If you want more info, run the line below.
# df_flight_info.loc[df_flight_info['ACT_AC_TYP_CD'].isin(['T7','U7'])]
# The two missing aircraft types are both 777. There are only 29 flight affected, so we can proceed without them when analysing by aircraft type.

array(['T7', 'U7'], dtype=object)

### More cleaning for columns redundancy

In [37]:
# drop redundant columns.

df_final = df_lounge_elig_flight_info_country_acft_typ.copy()

to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','ROUTE_x',
          'GMT_PLND_DEP_DT','ROUTE_y','DEP_STN_CD','ARR_STN_CD']

df_final = df_final.drop(columns = to_drop,)
df_final['GMT_UPLIFT_YR'] = [dts.year for dts in df_final['GMT_UPLIFT_DT']]

print(df_final.shape)
df_final.head(5)

(1534750, 26)


,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,3,319,M4,16:35:00,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,18:25:00,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,150.0,20.0,0.0,150.0,2019
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,14:40:00,FR,France,WEST EUROPE,EUROPE EXC UK,NB,119.0,16.0,0.0,119.0,2019
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,5,343,W9,16:15:00,EG,Egypt,NORTH AFRICA,AFRICA,WB,212.0,45.0,0.0,212.0,2019


<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [38]:
# your code here!

In [39]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [40]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count
0,NB,Not eligible,9741188
1,NB,Tier 1,53292
2,NB,Tier 2,744829
3,NB,Tier 3,2240579
4,WB,Not eligible,5730922
5,WB,Tier 1,236775
6,WB,Tier 2,391264
7,WB,Tier 3,1523611


In [41]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [42]:
# a look at the data
df_groupby_wb_nb_ttl

,WB_NB_CAT,pax_count_ttl
0,NB,12779888
1,WB,7882572


In [43]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [44]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,NB,Not eligible,9741188,12779888
1,NB,Tier 1,53292,12779888
2,NB,Tier 2,744829,12779888
3,NB,Tier 3,2240579,12779888
4,WB,Not eligible,5730922,7882572
5,WB,Tier 1,236775,7882572
6,WB,Tier 2,391264,7882572
7,WB,Tier 3,1523611,7882572


In [45]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [46]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [47]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_eligible%
0,NB,Not eligible,76.2%
1,NB,Tier 1,0.4%
2,NB,Tier 2,5.8%
3,NB,Tier 3,17.5%
4,WB,Not eligible,72.7%
5,WB,Tier 1,3.0%
6,WB,Tier 2,5.0%
7,WB,Tier 3,19.3%


In [48]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [49]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [50]:
# final table
df_groupby_wb_nb

pax_eligible%              
Lounge_eligibility_tier        Tier 1 Tier 2 Tier 3
WB_NB_CAT                                          
NB                               0.4%   5.8%  17.5%
WB                               3.0%   5.0%  19.3%

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [51]:
# your code here!

In [52]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count
0,5,HND,Not eligible,15825
1,5,HND,Tier 1,220
2,5,HND,Tier 2,888
3,5,HND,Tier 3,4469


In [53]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [54]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,pax_count_ttl
0,5,HND,21402
1,5,NRT,38378


In [55]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [56]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,5,HND,Not eligible,15825,21402
1,5,HND,Tier 1,220,21402
2,5,HND,Tier 2,888,21402
3,5,HND,Tier 3,4469,21402


In [57]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [58]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_eligible%
0,5,HND,Not eligible,73.9%
1,5,HND,Tier 1,1.0%
2,5,HND,Tier 2,4.1%
3,5,HND,Tier 3,20.9%


In [59]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [60]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [61]:
# final table
df_groupby_flt_no.head(4)

pax_eligible%              
Lounge_eligibility_tier                  Tier 1 Tier 2 Tier 3
OPERATING_FLT_NO DISCHARGE_STN_CD                            
5                HND                       1.0%   4.1%  20.9%
                 NRT                       3.6%   4.8%  19.1%
7                HND                       3.3%   5.2%  19.8%
9                BKK                       0.3%   4.2%  15.9%

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

## Additional data preparation.
Our analysis requires some additional data preparation, shown in this section.

Firstly, we would like to be able to split the data into 2019/23 data separately, which can be done by adding a year field into the table.

In [62]:
# First, convert uplift date to datetime object.

df_final['GMT_UPLIFT_DT'] = pd.to_datetime(df_final['GMT_UPLIFT_DT'])

# split year from planned departure date

df_final['GMT_UPLIFT_YR'] = df_final['GMT_UPLIFT_DT'].dt.year

In [63]:
# create a function to work out the OneWorld status category of each passenger group.
# The current dataset is not perfect: many BA status passengers are not correctly reported as having OneWorld status.

def status_column_creator(df):
    
    if 'STATUS' not in list(df.columns):
        df['STATUS'] = 'NA'
    
    ba_emerald_list = ['Gold','Gold For Life','Gold Guest List','Gold Guest List For Life','Premier']

    # calculate how many are of each OneWorld Status level. Order of operations is important, highest reported status prevails.
    df.loc[(df['BA_PAX_TIER'].isin(['None','']))|(df['ONEWORLD_TIER']==''),'STATUS']='None'
    df.loc[(df['BA_PAX_TIER']=='Silver')|(df['ONEWORLD_TIER']=='SAPP'),'STATUS']='Sapphire'
    df.loc[(df['BA_PAX_TIER'].isin(ba_emerald_list))|(df['ONEWORLD_TIER']=='EMER'),'STATUS']='Emerald'

    # set BA column only. If blank, we want it to read none.
    df.loc[(df['BA_PAX_TIER']==''),'BA_PAX_TIER']='None'
    
    return df

In [64]:
# run this function with our dataset.
df_final = status_column_creator(df_final)

In [65]:
# Create a function to work out why a passenger has their tier of lounge access.
# E.g. If a passenger is BA Silver and travels in First, they have Tier 1 lounge access from their travel cabin.
# E.g. If a OneWorld Emerald is travelling in World Traveller Plus, they have Tier 2 lounge access from their status.
# E.g. If a OneWorld Sapphire is travelling in Club Europe, they have Tier 3 lounge access from both their status and their cabin.
# Note: the original lounge eligibility dataset only uses Travel Cabin to calculate lounge eligibility, not booked cabin. This affects e.g. downgrades.

def tier_source_finder(df):

    # create two columns indicating whether pax sources lounge eligibility from cabin and/or status. Default to No.
    df['SOURCE_CABIN_IND'] = 'N'
    df['SOURCE_STATUS_IND'] = 'N'
    
    # if pax in Tier 1, are they flying in F? If so set cabin source indicator to Yes (Y).
    tier_1_cabin_y = (df['Lounge_eligibility_tier']=='Tier 1') & ((df['TRAVEL_CABIN_CD']=='F'))
    df.loc[tier_1_cabin_y,['SOURCE_CABIN_IND']] = 'Y'
    
    # if they are in Tier 1, do they have the status for Tier 1 lounge access. If so, set cabin source indicator to Yes (Y).

    tier_1_status_list = ['Gold Guest List','Gold Guest List For Life','Premier']
    tier_1_status_y = (df['Lounge_eligibility_tier']=='Tier 1') & ((df['BA_PAX_TIER'].isin(tier_1_status_list)))
    df.loc[tier_1_status_y,['SOURCE_STATUS_IND']] = 'Y'
    
    # if they are in Tier 2, do they have the status for Tier 2 lounge access, either from BA or OneWorld.
    # We will not use the STATUS column. This is Jamie's custom function, best not to use in case that doesn't make it to final notebook. Instead, check both BA and OneWorld Tiers.
    
    tier_2_status_list = ['Gold','Gold For Life','EMER']
    tier_2_status_y = (df['Lounge_eligibility_tier']=='Tier 2') & ((df['BA_PAX_TIER'].isin(tier_2_status_list)|(df['ONEWORLD_TIER'].isin(tier_2_status_list))))
    df.loc[tier_2_status_y,['SOURCE_STATUS_IND']] = 'Y'
    
    # If they are in Tier 3, do they have the cabin for Tier 3 lounge access?
    tier_3_cabins_list = ['C','J']
    tier_3_cabins_y = (df['Lounge_eligibility_tier']=='Tier 3') & ((df['TRAVEL_CABIN_CD'].isin(tier_3_cabins_list)))
    df.loc[tier_3_cabins_y,['SOURCE_CABIN_IND']] = 'Y'
    
    # If they are in Tier 3, do they have the status for Tier 3 access, either BA or OneWorld?
    
    tier_3_status_list = ['Silver','SAPP']
    tier_3_status_y = (df['Lounge_eligibility_tier']=='Tier 3') & ((df['BA_PAX_TIER'].isin(tier_3_status_list)|(df['ONEWORLD_TIER'].isin(tier_3_status_list))))
    df.loc[tier_3_status_y,['SOURCE_STATUS_IND']] = 'Y'
    
    return df

In [66]:
# run this function.
df_final = tier_source_finder(df_final)

Add haul data

Hauls vary by both location and by date. Some routes have switched haul relatively recently e.g. Amman, Cairo.
Some long haul routes are shorter than short-haul routes e.g. Tel Aviv is shorter than Amman.

A valid way of categorising flights is by selling classes: only long haul flights sell First (F) and World Traveller Plus (W). Short haul flights sell Club as C and long haul flights sell Club as J.


In [67]:
# First, group flights by Skew ID and passengers in each cabin per Skew ID. Fill missing values with zero.

def add_haul(df):

    grouped_df_destinations_dates = df.groupby(['Skew_Id','TRAVEL_CABIN_CD']).agg({'pax':'sum'}).reset_index()
    pivot_destination_dates = grouped_df_destinations_dates.pivot(index='Skew_Id',columns='TRAVEL_CABIN_CD',values='pax').reset_index()
    pivot_destination_dates.fillna(0,inplace=True)
    pivot_destination_dates['HAUL'] = pd.NaT

    # work out if long or short haul. The data are clean, there are none selling in both C and any of F, J and W.
    long_haul_ind = (pivot_destination_dates['F']>0)|(pivot_destination_dates['J']>0)|(pivot_destination_dates['W']>0) # any number of seats sold in First, Club World or WTP.
    short_haul_ind = (pivot_destination_dates['C']>0) # any number of seats sold in Club Europe means flight is short haul.
    pivot_destination_dates.loc[long_haul_ind,['HAUL']] = 'L'
    pivot_destination_dates.loc[~long_haul_ind,['HAUL']] = 'S'
    
    df_merged = pd.merge(df,pivot_destination_dates[['Skew_Id','HAUL']],on='Skew_Id')
    
    return df_merged

df_final = add_haul(df_final)

In [68]:
df_final.head(10)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,STATUS,SOURCE_CABIN_IND,SOURCE_STATUS_IND,HAUL
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
1,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,,Tier 1,1,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
2,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,None,,Not eligible,68,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,None,N,N,S
3,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Gold Guest List,EMER,Tier 1,4,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
4,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Silver,SAPP,Tier 3,6,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Sapphire,N,Y,S
5,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Premier,EMER,Tier 1,1,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
6,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,Silver,,Tier 3,1,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Sapphire,N,Y,S
7,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold Guest List,EMER,Tier 1,1,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
8,2023-08-05618,BA,618,2023-08-05,LHR,OLB,M,M,None,EMER,Tier 2,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S
9,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Silver,SAPP,Tier 3,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Sapphire,Y,Y,S


In [71]:
# TEMP

# EXPORT TO CSV (VERY BIG)
# CSV needs to be moved after production - cannot be pushed to github and it will mess everything up if you sync with it in folder.

df_final.to_csv('df_final_20231102.csv')

### Group by functions

Very useful for later

In [69]:
def group_pivot(df, list_groupby):
    Tiers = ['Not eligible','Tier 1','Tier 2','Tier 3']

    # Use square brackets for column names with spaces
    df_grouped = df.groupby(list_groupby).agg({
        ['Not eligible']: 'sum',
        ['Tier 1']: 'sum',
        ['Tier 2']: 'sum',
        ['Tier 3']: 'sum',
        ['TOTAL_PAX']: 'sum'
    }).reset_index()

    for t in Tiers:
        df_grouped[t] = (df_grouped[t] / df_grouped['TOTAL_PAX']) * 100
        df_grouped[t] = df_grouped[t].map('{:,.2f}%'.format)

    return df_grouped

def group_pivot_to_lookup(df):
    return df.set_index(list(df.columns))

In [70]:
df_final.head(1)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,STATUS,SOURCE_CABIN_IND,SOURCE_STATUS_IND,HAUL
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,14:25:00,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,113.0,21.0,0.0,113.0,2023,Emerald,N,Y,S


### PLEASE IGNORE ME

### Group output 1: Differences between 2019 and 2023

In [71]:
# conduct an initial analysis of the differences in tier by year.

# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year = df_final.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year

,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_count
0,2019,Not eligible,8147159
1,2019,Tier 1,196564
2,2019,Tier 2,635765
3,2019,Tier 3,1996226
4,2023,Not eligible,7372440
5,2023,Tier 1,93555
6,2023,Tier 2,501044
7,2023,Tier 3,1770087


In [72]:
# any additional year analysis.

In [ ]:
# # Creation of final pivot table with all relevant info.
# # Numbers split long and short haul with lounge elig attached.

# # add GMT Uplift Yr if needed

# df_final_col_list = list(df_final.columns)
# pivot_table_drop = ['BA_PAX_TIER','ONEWORLD_TIER','Lounge_eligibility_tier','pax','BOOKED_CABIN_CD','TRAVEL_CABIN_CD']

# pivot_df_final = pd.pivot_table(df_final, values='pax', index=list(df_final.drop(pivot_table_drop,axis=1).columns),
#                                 columns='Lounge_eligibility_tier',aggfunc='sum',fill_value=0).reset_index()
# pivot_df_final['TOTAL_PAX'] = pivot_df_final['Not eligible']+pivot_df_final['Tier 1']+pivot_df_final['Tier 2']+pivot_df_final['Tier 3']
# pivot_df_final.head(5)



**Create new column 'Status' based on BA and Oneworld Status.**

In [ ]:
# define the source of the pax status
v2_df_final_status = df_final_status.copy()
v2_df_final_status.head(10)

### Create and run function to work out why a passenger has lounge access. Now moved

In [ ]:
df_final_col_list = list(df_final.columns)
pivot_table_drop = ['BA_PAX_TIER','ONEWORLD_TIER','Lounge_eligibility_tier','pax','BOOKED_CABIN_CD','TRAVEL_CABIN_CD','STATUS']

pivot_df_final_status_OW = pd.pivot_table(df_final_status, values='pax', index=list(df_final_status.drop(pivot_table_drop,axis=1).columns),
                                columns='STATUS',aggfunc='sum',fill_value=0).reset_index()
pivot_df_final_status_OW.rename(columns={'None':'None_OW'},inplace=True)
pivot_df_final_status_OW.head(5)

In [ ]:
df_final_col_list = list(df_final.columns)
pivot_table_drop = ['BA_PAX_TIER','ONEWORLD_TIER','Lounge_eligibility_tier','pax','BOOKED_CABIN_CD','TRAVEL_CABIN_CD','STATUS']

pivot_df_final_status = pd.pivot_table(df_final_status, values='pax', index=list(df_final_status.drop(pivot_table_drop,axis=1).columns),
                                columns=['BA_PAX_TIER'],aggfunc='sum',fill_value=0).reset_index()
pivot_df_final_status.head(5)

In [ ]:
# FINAL OUTPUT
# NOTE: The values are not cumulative ie. the OneWorld and BA Tiers must be treated separately. OneWorld Tiers are inclusive of BA tiers.
pivot_df_final_status= pivot_df_final_status.merge(pivot_df_final_status_OW[['Skew_Id','Emerald','None_OW','Sapphire']]
                           , on='Skew_Id',how='left')
pivot_df_final_status.head(5)

In [ ]:
pivot_df_final_status.sort_values(by='Premier',ascending=False).head(20)

In [ ]:
# export dataframe with status and pax eligible for tier 1, 2 and 3
pivot_df_final_merged = pivot_df_final_status.merge(pivot_df_final[['GMT_UPLIFT_DT','Skew_Id','GMT_PLND_DEP_TIME','Not eligible','Tier 1','Tier 2','Tier 3','TOTAL_PAX']],on=['Skew_Id','GMT_PLND_DEP_TIME','GMT_UPLIFT_DT'],how='left')

In [ ]:
# just check no rows were lost.
print(pivot_df_final_merged.shape)
print(pivot_df_final_status.shape)
print(pivot_df_final_status_OW.shape)
# should all be 130976.

# check this is the same as the unique skew id in the original dataset.
print(df_lounge_eligibility['Skew_Id'].nunique())
df_lounge_eligibility[~df_lounge_eligibility['Skew_Id'].isin(list(pivot_df_final_merged['Skew_Id'].unique()))].sort_values(by='Skew_Id').shape


In [ ]:
# add percentages
pivot_df_final_merged.head(1)

In [ ]:
#pivot_df_final_merged.to_csv('../../scragg_pivot_df_final_merged.csv')
#pivot_df_final_merged.head(1)

--

### JAMIE ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pivot_df_final_merged.head(5)

In [ ]:
final_final_pivot_all = pivot_df_final_merged.copy()

In [ ]:
final_final_pivot_all[['FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY']] = final_final_pivot_all[['FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY']].astype(int)
final_final_pivot_all['TOTAL_CAPACITY'] = (final_final_pivot_all[['FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY']].sum(axis=1))
final_final_pivot_all['LOAD_PERCENTAGE'] = 100*(final_final_pivot_all['TOTAL_PAX']/final_final_pivot_all['TOTAL_CAPACITY'])
final_final_pivot_all.loc[final_final_pivot_all['LOAD_PERCENTAGE']>105].shape

In [ ]:
final_final_pivot_all.head(1)

In [ ]:
# pivot_df_final_status_OW = pd.pivot_table(df_final_status, values='pax', index=list(df_final_status.drop(pivot_table_drop,axis=1).columns),
#                                 columns='STATUS',aggfunc='sum',fill_value=0).reset_index()

In [ ]:
# route_groups = group_pivot(final_final_pivot_all,['DISCHARGE_STN_CD'])

route_groups_mine = final_final_pivot_all.groupby('DISCHARGE_STN_CD').agg(
    
    NOT_ELIGIBLE = ('Not eligible',sum),
    TIER_3 = ('Tier 3',sum),
    TIER_2 = ('Tier 2',sum),
    TIER_1 = ('Tier 1',sum), 
    TOTAL_PAX = ('TOTAL_PAX',sum)).reset_index()



In [73]:
# Working version
def group_pivot(df, list_groupby):
    Tiers = ['Not eligible','Tier 1','Tier 2','Tier 3']

    # Use square brackets for column names with spaces
    df_grouped = df.groupby(list_groupby).agg({
        'Not eligible': 'sum',
        'Tier 1': 'sum',
        'Tier 2': 'sum',
        'Tier 3': 'sum',
        'TOTAL_PAX': 'sum'
    }).reset_index()

    for t in Tiers:
        df_grouped[t] = (df_grouped[t] / df_grouped['TOTAL_PAX']) * 100
        df_grouped[t] = df_grouped[t].map('{:,.2f}%'.format)

    return df_grouped

def group_pivot_to_lookup(df):
    return df.set_index(list(df.columns))

In [74]:
final_final_pivot_all.head(1)

NameError: name 'final_final_pivot_all' is not defined

In [75]:
route_groups = group_pivot(final_final_pivot_all,'DISCHARGE_STN_CD')

NameError: name 'final_final_pivot_all' is not defined

In [ ]:
route_groups.head(10)

In [ ]:
aircraft_types_v1 = group_pivot(final_final_pivot_all,['IATA_AC_TYP_CD','CORP_GEOG_CTRY_GRP_NM'])
aircraft_types_v1_lookup = group_pivot_to_lookup(aircraft_types_v1)
aircraft_types_v1_lookup.sort_values(['IATA_AC_TYP_CD','Tier 1'],ascending=[True,False])

In [ ]:
# aircraft_types_v1 = group_pivot(final_final_pivot_all,['IATA_AC_TYP_CD','ACT_AC_TYP_CD'])

# aircraft_types_v1 = aircraft_types_v1.merge(df_acft_typ[-'IATA_AC_TYP_CD'],on='ACT_AC_TYP_CD',how='left')
# #aircraft_types_v1.sort_values(['Tier 1','IATA_AC_TYP_CD'],ascending=[False,True])
# #aircraft_types_v1.sort_values('Tier 1',ascending=False)
# #aircraft_types_v1 = group_pivot_to_lookup(aircraft_types_v1)
# aircraft_types_v1

In [ ]:
final_final_pivot_all.head(5)

In [ ]:
final_final_pivot_all['GMT_PLND_DEP_HOUR'] = pd.to_datetime(final_final_pivot_all
                                                           ['GMT_PLND_DEP_TIME'],format='%H:%M:%S').dt.hour


In [ ]:
# visualise the wide/narrow body split per hour.

pivot_wb_nb_hour = pd.pivot_table(final_final_pivot_all,index='GMT_PLND_DEP_HOUR'
                                 ,columns='WB_NB_CAT',aggfunc='size',fill_value=0)
pivot_wb_nb_hour['Total'] = pivot_wb_nb_hour['NB'] + pivot_wb_nb_hour['WB']

pivot_wb_nb_hour['Narrow'] = (pivot_wb_nb_hour['NB']/pivot_wb_nb_hour['Total'])*100
pivot_wb_nb_hour['Wide'] = (pivot_wb_nb_hour['WB']/pivot_wb_nb_hour['Total'])*100

ax=pivot_wb_nb_hour[['Narrow','Wide']].plot(kind='bar'
                                           ,stacked=True
                                           ,figsize=(10,6))
# Customize the chart
ax.set_xlabel('Hour of Departure')
ax.set_ylabel('Percentage')
ax.set_title('Percentage of Narrow and Wide-Body Flights by Hour of Departure')
plt.legend(title='Aircraft Type')
plt.show()

In [ ]:
final_final_pivot_all.head(1)

In [ ]:
# visualise the tier proportions per hour
fig, ax = plt.subplots(figsize=(10, 6))


tier_list = ['Not eligible','Tier 3','Tier 2','Tier 1']

pivot_tiers_hours = final_final_pivot_all.groupby(['GMT_PLND_DEP_HOUR'])[tier_list].sum()
pivot_tiers_hours['Total'] = pivot_tiers_hours.sum(axis=1)

for tier in tier_list:
    pivot_tiers_hours[f'{tier}'] = (pivot_tiers_hours[tier]/pivot_tiers_hours['Total'])*100
    
ax=pivot_tiers_hours[[f'{tier}' for tier in tier_list]].plot(kind='bar',stacked=True,ax=ax)

# add total pax line
ax2=ax.twinx()

pivot_tiers_hours['Average pax'] = (final_final_pivot_all.groupby(['GMT_PLND_DEP_HOUR'])['TOTAL_PAX'].sum())/len(final_final_pivot_all['GMT_UPLIFT_DT'].unique())
pivot_tiers_hours['Average pax'].plot(kind='line',linestyle='-',color='black',marker='*',ax=ax2,label='Total Pax')

# Customize the line graph
ax2.set_ylabel('Average Pax')
ax2.legend(loc='upper right')
ax.legend(loc='lower right')



plt.show()

In [ ]:
pivot_tiers_hours.head(10)

In [ ]:
# EXPORT TO CSV
# final_final_pivot_all.to_csv('../../final_final_pivot_all.csv')

### Linear Regression

### Compare by Year

In [ ]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year_ttl

In [ ]:
df_groupby_year = pd.merge(df_groupby_year,
                            df_groupby_year_ttl,
                            on = ['GMT_UPLIFT_YR'],
                            how = 'left'
                           )

df_groupby_year

In [ ]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_year['pax_eligible%'] = (df_groupby_year['pax_count'] / df_groupby_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_year['pax_eligible%'] = df_groupby_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [ ]:
df_groupby_year = df_groupby_year[df_groupby_year['Lounge_eligibility_tier'] != "Not eligible"]



Not much difference

### Compare by location and year

In [ ]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_country_group_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR']

df_groupby_country_group_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_country_group_year = pd.merge(df_groupby_country_group_year,
                                         df_groupby_country_group_year_ttl,
                                         on = list_groupby,
                                         how = "left")

df_groupby_country_group_year = df_groupby_country_group_year[df_groupby_country_group_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_country_group_year['pax_eligible%'] = (df_groupby_country_group_year['pax_count'] / df_groupby_country_group_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_country_group_year['pax_eligible%'] = df_groupby_country_group_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_country_group_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_country_group_year.head(6)


In [ ]:
df_groupby_country_group_year = df_groupby_country_group_year.set_index(['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_country_group_year

### By continent

In [ ]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_cont_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR']

df_groupby_cont_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_cont_year = pd.merge(df_groupby_cont_year,
                                df_groupby_cont_year_ttl,
                                on = list_groupby,
                                how = "left")

df_groupby_cont_year = df_groupby_cont_year[df_groupby_cont_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_cont_year['pax_eligible%'] = (df_groupby_cont_year['pax_count'] / df_groupby_cont_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_cont_year['pax_eligible%'] = df_groupby_cont_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_cont_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_cont_year.head(6)

In [ ]:
df_groupby_cont_year = df_groupby_cont_year.set_index(['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_cont_year

### By aircraft

In [ ]:
list_groupby = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_acft = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                mean_pax_count = ('pax','mean')).reset_index()

df_groupby_acft['mean_pax_count'] = df_groupby_acft['mean_pax_count'].map('{:,.1f}'.format)

df_groupby_acft = df_groupby_acft[df_groupby_acft['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_acft = df_groupby_acft.set_index(list_groupby,drop = True).unstack('Lounge_eligibility_tier')

df_groupby_acft.fillna(0, inplace=True)

df_groupby_acft


<br><br><br>

In [ ]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>
- ... (your answer here)

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

In [ ]:
# your code here

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.